# 비정규성 측정 (Measuring Non-normality)

이번 실습 시간에는 `scipy.stats` 모듈에서 사용가능한 왜도와 첨도에 대한 코드를 개발한 뒤, 이를 헤지 펀드 지수 수익률에 적용해 볼 것입니다.

또한 `scipy.stats` 모듈을 사용하여 데이터의 정규성에 대한 [**자크-베라(Jarque-Bera)**](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test) 테스트를 해보고, 이를 다른 수익률 시리즈에 적용하는 방법도 살펴보겠습니다.

우선, 아래의 소스코드를 `port_opt_toolkit.py` 파일에 추가합니다.

함수
1. `get_hfi_returns()` : 헤지펀드 지수 수익률 데이터 불러오기

```python
def get_hfi_returns():
    """
    Load and format the EDHEC Hedge Fund Index Returns
    """
    hfi = pd.read_csv("data/edhec-hedgefundindices.csv",
                      header=0, index_col=0, parse_dates=True)
    hfi = hfi/100
    hfi.index = hfi.index.to_period('M')
    return hfi
```


In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import port_opt_toolkit as potk
hfi = potk.get_hfi_returns()
hfi.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
1997-01,0.0119,0.0393,0.0178,0.0791,0.0189,0.0213,0.0191,0.0573,0.0281,0.0150,0.0180,-0.0166,0.0317
1997-02,0.0123,0.0298,0.0122,0.0525,0.0101,0.0084,0.0122,0.0175,-0.0006,0.0034,0.0118,0.0426,0.0106
1997-03,0.0078,-0.0021,-0.0012,-0.0120,0.0016,-0.0023,0.0109,-0.0119,-0.0084,0.0060,0.0010,0.0778,-0.0077
1997-04,0.0086,-0.0170,0.0030,0.0119,0.0119,-0.0005,0.0130,0.0172,0.0084,-0.0001,0.0122,-0.0129,0.0009
1997-05,0.0156,-0.0015,0.0233,0.0315,0.0189,0.0346,0.0118,0.0108,0.0394,0.0197,0.0173,-0.0737,0.0275


## 왜도 (Skewness)

직관적으로, 음의 스큐는 수익률이 정규 분포를 가정한 경우보다 더 많은 음의 수익률을 얻는다는 것을 의미합니다.

이에 대해 생각하는 또 다른 방법은 만약 수익률이 정규 분포를 따르는 경우 평균과 중앙값이 매우 가깝다는 것입니다.

하지만 음으로 치우친 경우 기대값, 즉 평균은 중앙값보다 작습니다. 반대로 양으로 치우친 경우 기대값(평균)이 중앙값보다 큽니다.

In [2]:
pd.concat([hfi.mean(), hfi.median(), hfi.mean() > hfi.median()], axis=1)

,0,1,2
Convertible Arbitrage,0.005508,0.0065,False
CTA Global,0.004074,0.0014,True
Distressed Securities,0.006946,0.0089,False
Emerging Markets,0.006253,0.0096,False
Equity Market Neutral,0.004498,0.0051,False
Event Driven,0.006344,0.0084,False
Fixed Income Arbitrage,0.004365,0.0055,False
Global Macro,0.005403,0.0038,True
Long/Short Equity,0.006331,0.0079,False
Merger Arbitrage,0.005356,0.0060,False


이제 왜도를 계산하는 코드를 만들어 보겠습니다.

왜도의 공식은 다음과 같습니다.

$$ S(R) = \frac{E[ (R-E(R))^3 ]}{\sigma_R^3} $$


In [12]:
def skewness(r):
    """
    Alternative to scipy.stats.skew()
    Computes the skewness of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**3).mean()
    return exp/sigma_r**3

In [13]:
skewness(hfi).sort_values()

Fixed Income Arbitrage   -3.940320
Convertible Arbitrage    -2.639592
Equity Market Neutral    -2.124435
Relative Value           -1.815470
Event Driven             -1.409154
Merger Arbitrage         -1.320083
Distressed Securities    -1.300842
Emerging Markets         -1.167067
Long/Short Equity        -0.390227
Funds Of Funds           -0.361783
CTA Global                0.173699
Short Selling             0.767975
Global Macro              0.982922
dtype: float64

우리가 구한 값이 맞는지 확인하기 위해, `scipy.stats`의 왜도 함수를 사용해보겠습니다.

In [18]:
import scipy.stats
scipy.stats.skew(hfi)

array([-2.63959223,  0.17369864, -1.30084204, -1.16706749, -2.12443538,
       -1.40915356, -3.94032029,  0.98292188, -0.39022677, -1.32008333,
       -1.81546975,  0.76797484, -0.36178308])

In [20]:
skewness(hfi)

Convertible Arbitrage    -2.639592
CTA Global                0.173699
Distressed Securities    -1.300842
Emerging Markets         -1.167067
Equity Market Neutral    -2.124435
Event Driven             -1.409154
Fixed Income Arbitrage   -3.940320
Global Macro              0.982922
Long/Short Equity        -0.390227
Merger Arbitrage         -1.320083
Relative Value           -1.815470
Short Selling             0.767975
Funds Of Funds           -0.361783
dtype: float64

위에서 구현한 왜도 함수를 `port_opt_toolkit.py`에 추가해줍니다.

마지막으로, 무작위적으로 생성된 수익률 시리즈에서 기대할 수 있는 왜도를 살펴보겠습니다. 넘파이의 랜덤 정규 분포 생성기를 사용하여 헤지 펀드 데이터와 동일한 수의 수익률 데이터 포인트를 생성해 보겠습니다.

In [21]:
hfi.shape

(263, 13)

In [22]:
import numpy as np
normal_rets = np.random.normal(0, 0.15, (263, 1))

In [23]:
normal_rets.mean(), normal_rets.std()

(-0.019175280283941873, 0.15971610038470918)

In [24]:
potk.skewness(normal_rets)

-0.09868435557305841

# 첨도 (Kurtosis)

직관적으로 첨도는 어떤 분포에서 꼬리 부분의 "두꺼움"을 측정합니다. 정규 분포의 첨도는 3이므로 수익률의 첨도가 3보다 작으면 꼬리가 더 얇은 경향이 있고 첨도가 3보다 크면 분포가 더 두꺼운 꼬리를 갖습니다.

첨도의 공식은 다음과 같습니다.

$$ K(R) = \frac{E[ (R-E(R))^4 ]}{\sigma_R^4} $$

첨도의 공식은 왜도를 계산하는 방식과 매우 유사하기 때문에, 이를 복사하여 붙여넣은 다음 3승을 4승으로 바꾸어주기만 하면 됩니다.

In [26]:
potk.kurtosis(hfi)

Convertible Arbitrage     23.280834
CTA Global                 2.952960
Distressed Securities      7.889983
Emerging Markets           9.250788
Equity Market Neutral     17.218555
Event Driven               8.035828
Fixed Income Arbitrage    29.842199
Global Macro               5.741679
Long/Short Equity          4.523893
Merger Arbitrage           8.738950
Relative Value            12.121208
Short Selling              6.117772
Funds Of Funds             7.070153
dtype: float64

이를 `scipy.stats`의 `kurtosis()` 메서드와 비교해보겠습니다.

In [27]:
scipy.stats.kurtosis(hfi)

array([20.28083446, -0.04703963,  4.88998336,  6.25078841, 14.21855526,
        5.03582817, 26.84219928,  2.74167945,  1.52389258,  5.73894979,
        9.12120787,  3.11777175,  4.07015278])

출력된 숫자들은 모두 우리가 계산한 숫자보다 3이 모자랍니다. 앞에서 언급했듯이 정규 분포 숫자 시리즈의 예상 첨도는 3입니다. `scipy.stats`는 그냥 첨도가 아닌 3보다 초과된 **초과 첨도(Excess Kurtosis)**를 반환합니다. 우리가 생성한 임의의 일반 숫자에 적용하여 이를 확인할 수 있습니다.

In [28]:
scipy.stats.kurtosis(normal_rets)

array([-0.47243044])

In [30]:
potk.kurtosis(normal_rets)

2.5275695611324185

## 정규성 테스트를 위한 자크-베라 검정

`scipy.stats` 모듈에는 일련의 숫자에 대해 **자크-베라(Jarque-Bera)** 테스트를 실행하는 `jarque-bera()` 함수가 포함되어 있습니다. 이를 정규 분포로부터 생성된 수익률 데이터에 적용해 보겠습니다.

In [31]:
scipy.stats.jarque_bera(normal_rets)

(2.8726714994678786, 0.23779751381197922)

The first number is the test statistic and the second number is the one we want. It represents the p-value for the hypothesis test. If you want to run the test at a 1% level of significance, you want this number to be greater than 0.01 to accept the hypothesis that the data is normally distributed, and if that number is less than 0.01 then you must reject the hypothesis of normality.

첫 번째 숫자는 검정 통계량(test statistic)이고 두 번째 숫자는 가설 검정에 대한 p-값(p-value)을 나타냅니다. 만약 유의 수준이 1%라면, p-값이 1%보다 큰 경우는 귀무 가설을 받아들이고, 그렇지 않다면 정규 분포라는 귀무 가설을 기각하게 됩니다.

이 경우 p-값이 0.01보다 큰 숫자를 얻었으므로 해당 숫자들이 무작위라는 가설을 받아들일 수 있습니다. 이제 이 자크-베라 검정을 다양한 헤지펀드 지수 수익률에 대해 시도해 보겠습니다.

In [32]:
scipy.stats.jarque_bera(hfi)

(25656.585999171326, 0.0)

어라, 위의 실행이 개별 지수에 대한 결과를 얻지 못한 이유는 대체 무엇일까요? 그것은 검정 구현이 각 열을 별도의 반환 집합으로 인식하여 처리할 만큼 똑똑하지 않기 때문입니다. 이를 수정하기 위해서는 자체적인 래퍼 함수를 작성할 수 있습니다. 여기서는 간단한 래퍼 함수를 작성하고 이 코드를 파이썬 모듈에 추가해보겠습니다.

```python
import scipy.stats
def is_normal(r, level=0.01):
    """
    Applies the Jarque-Bera test to determine if a Series is normal or not
    Test is applied at the 1% level by default
    Returns True if the hypothesis of normality is accepted, False otherwise
    """
    statistic, p_value = scipy.stats.jarque_bera(r)
    return p_value > level
```

In [33]:
potk.is_normal(normal_rets)

True

또한 이 문제를 처리할 수 있는 몇 가지 다른 방법들이 있습니다. 첫 번째 방법은 데이터프레임에서 `.aggregate` 메서드를 사용하는 것입니다. 이 메서드는 함수를 인수로 받아 해당 함수를 각 열에 적용합니다.

In [35]:
hfi.aggregate(potk.is_normal)

Convertible Arbitrage     False
CTA Global                 True
Distressed Securities     False
Emerging Markets          False
Equity Market Neutral     False
Event Driven              False
Fixed Income Arbitrage    False
Global Macro              False
Long/Short Equity         False
Merger Arbitrage          False
Relative Value            False
Short Selling             False
Funds Of Funds            False
dtype: bool

혹은 정규성을 테스트하기 위해 통일된 인터페이스를 갖도록 래퍼 함수를 수정하여 문제를 해결할 수도 있습니다.

```python
import scipy.stats
def is_normal(r, level=0.01):
    """
    Applies the Jarque-Bera test to determine if a Series is normal or not
    Test is applied at the 1% level by default
    Returns True if the hypothesis of normality is accepted, False otherwise
    """
    if isinstance(r, pd.DataFrame):
        return r.aggregate(is_normal)
    else:
        statistic, p_value = scipy.stats.jarque_bera(r)
        return p_value > level
```

In [42]:
import pandas as pd
isinstance(hfi, pd.DataFrame)

True

In [39]:
potk.is_normal(hfi)

Convertible Arbitrage     False
CTA Global                 True
Distressed Securities     False
Emerging Markets          False
Equity Market Neutral     False
Event Driven              False
Fixed Income Arbitrage    False
Global Macro              False
Long/Short Equity         False
Merger Arbitrage          False
Relative Value            False
Short Selling             False
Funds Of Funds            False
dtype: bool

In [40]:
potk.is_normal(normal_rets)

True

## 대형주 포트폴리오와 소형주 포트폴리오에 대한 정규성 검정

대형주 포트폴리오와 소형주 포트폴리오 중 위의 정규성 가설을 통과하는 것이 있는지 살펴보겠습니다.

In [44]:
ffme = potk.get_ffme_returns()
potk.skewness(ffme)

SmallCap    4.410739
LargeCap    0.233445
dtype: float64

In [45]:
potk.kurtosis(ffme)

SmallCap    46.845008
LargeCap    10.694654
dtype: float64

In [46]:
potk.is_normal(ffme)

SmallCap    False
LargeCap    False
dtype: bool